<a href="https://colab.research.google.com/github/Talha1818/Machine-Learning-Python-Notebooks/blob/master/FastText_Text_Classification_TransData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Annotation Folder/Text_dataset_27Feb_Merge_with_21Feb.csv")
df.head()

,serial_number,data,state,county,target
0,16070,"HAD DONFIELD - CITY ABSTRACT, L 1 LBS 1 OF 1\n...",NJ,camden,15
1,16071,re ‘epee (o| {253 / PNCBAD i c 3-5/310\noe G P...,NJ,camden,15
2,16072,|\nACKNOWLEDGMENT OF NO SURVEY COVERAGE\nSTATE...,NJ,camden,72
3,16073,"City Abstract, LLC ALTA Buyer's Settlement Sta...",NJ,camden,11
4,16074,Description\ntit Credit\nTitle - Overnight Fee...,NJ,camden,11


In [ ]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
import re


# Define regex patterns
punct_pattern = r'[^\w\s]'
num_pattern = r'\d+'
special_pattern = r'[^A-Za-z0-9\s]'

# Define cleaning function
def clean_text(text):
    text = re.sub(r'\\n|\n', ' ', text)
    # remove links
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'\x00|\\x00', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(punct_pattern, '', text)
    # Remove integers
    text = re.sub(num_pattern, '', text)
    # Remove special characters
    text = re.sub(special_pattern, '', text)
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()
    # Remove nextline
    text = re.sub(r'\n', ' ', text)
    # Remove words with length 1
    text = ' '.join(word for word in text.split() if len(word) > 2)
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = df[df['data'].notna()]
df = df[df['state'].notna()]
df = df[df['county'].notna()]
df = df[df['target'].notna()]
# Apply cleaning function to text feature
df['clean_text'] = df['data'].apply(clean_text)

In [ ]:
# df['clean_text'] = "<START> " + df['clean_text'] + " <END> <DOCUMENT>"

In [ ]:
df = df.rename({'target':'token'}, axis = 1)
df.head()

,serial_number,data,state,county,token,clean_text
0,16070,"HAD DONFIELD - CITY ABSTRACT, L 1 LBS 1 OF 1\n...",NJ,camden,15,donfield city abstract lbs bash aonon avec shi...
1,16071,re ‘epee (o| {253 / PNCBAD i c 3-5/310\noe G P...,NJ,camden,15,epee pncbad pnc bank national association cash...
2,16072,|\nACKNOWLEDGMENT OF NO SURVEY COVERAGE\nSTATE...,NJ,camden,72,acknowledgment survey coverage state applicati...
3,16073,"City Abstract, LLC ALTA Buyer's Settlement Sta...",NJ,camden,11,city abstract llc alta buyers settlement state...
4,16074,Description\ntit Credit\nTitle - Overnight Fee...,NJ,camden,11,description tit credit title overnight fee cit...


In [ ]:
df1 = df['token'].value_counts().to_frame().reset_index().rename(columns={'index':'token','token':'count'})
df_100 = df1[df1['count']>10]['token'].tolist()
df_new = df[df['token'].isin(df_100)]
len(df_new['token'].unique())

117

In [ ]:
df = df_new.copy()

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# # Create label encoder object
# label_encoder = LabelEncoder()

# # Fit label encoder and transform labels
# df['token'] = label_encoder.fit_transform(df['token'])
# df['token'].value_counts()

In [ ]:
df.head()

,serial_number,data,state,county,token,clean_text
0,16070,"HAD DONFIELD - CITY ABSTRACT, L 1 LBS 1 OF 1\n...",NJ,camden,15,donfield city abstract lbs bash aonon avec shi...
1,16071,re ‘epee (o| {253 / PNCBAD i c 3-5/310\noe G P...,NJ,camden,15,epee pncbad pnc bank national association cash...
2,16072,|\nACKNOWLEDGMENT OF NO SURVEY COVERAGE\nSTATE...,NJ,camden,72,acknowledgment survey coverage state applicati...
3,16073,"City Abstract, LLC ALTA Buyer's Settlement Sta...",NJ,camden,11,city abstract llc alta buyers settlement state...
4,16074,Description\ntit Credit\nTitle - Overnight Fee...,NJ,camden,11,description tit credit title overnight fee cit...


In [ ]:
df['label_token'] = "__label__"+df['token']

In [ ]:
df['token_description'] = df['label_token'] + " " + df['clean_text']
df[['token_description']].head()

,token_description
0,__label__15 donfield city abstract lbs bash ao...
1,__label__15 epee pncbad pnc bank national asso...
2,__label__72 acknowledgment survey coverage sta...
3,__label__11 city abstract llc alta buyers sett...
4,__label__11 description tit credit title overn...


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=2022
)

In [ ]:
train_df.shape, test_df.shape

((8459, 8), (2115, 8))

In [ ]:
train_df.to_csv("train.csv", columns=['token_description'], index=False, header=False)
test_df.to_csv("test.csv", columns=['token_description'], index=False, header=False)

In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import fasttext

# Train the model
model = fasttext.train_supervised('train.csv')

In [ ]:
# Test the model
result = model.test('test.csv')
result
# print('Test accuracy:', result[1])
# print('Test accuracy:', result[])
# print('Test accuracy:', result[1])


(2115, 0.47423167848699765, 0.47423167848699765)

In [ ]:
# Predict labels for new data
text = 'stateof new jersey county camden subscribed sworn february pyeliza stasi affiant name weblarne ais lor maureen woollam notarys signatore sue notary public state new jersey commission expires maween woow notarys printed typed stamped name seal seal commission expires arr namsc docmagic page'
predicted_label = model.predict(text)
predicted_label

(('__label__2',), array([0.2274082]))

In [ ]:
df['clean_text'].iloc[45]

'<START> stateof new jersey county camden subscribed sworn february pyeliza stasi affiant name weblarne ais lor maureen woollam notarys signatore sue notary public state new jersey commission expires maween woow notarys printed typed stamped name seal seal commission expires arr namsc docmagic page <END> <DOCUMENT>'

In [ ]:
df['token'].iloc[45]


'21'

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Annotation Folder/Final_Text_Dataset_10_March_2023.csv")
df = df[['clean_text', 'token']]
df['token'] = df['token'].astype(int)
df.head()

,clean_text,token
0,ong xmwor nnupoo sko uuuuuuu aneaf uuuuu tssar...,15
1,gem cashiers check remiter salvador carrasco o...,15
2,city abstract llc alta buyers final settlement...,11
3,description tile platform fee tmp ciyabstrack ...,11
4,city abstract llc alta sellers settlement stat...,13


In [ ]:
# df['clean_text'] = "<START> " + df['clean_text'] + " <END> <DOCUMENT>"
df1 = df['token'].value_counts().to_frame().reset_index().rename(columns={'index':'token','token':'count'})
df_100 = df1[df1['count']>20]['token'].tolist()
df_new = df[df['token'].isin(df_100)]
len(df_new['token'].unique())

216

In [ ]:
df['label_token'] = "__label__"+df['token'].astype(str)
df['token_description'] = df['label_token'] + " " + df['clean_text']
df[['token_description']].head()

,token_description
0,__label__15 ong xmwor nnupoo sko uuuuuuu aneaf...
1,__label__15 gem cashiers check remiter salvado...
2,__label__11 city abstract llc alta buyers fina...
3,__label__11 description tile platform fee tmp ...
4,__label__13 city abstract llc alta sellers set...


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=2022
)

train_df.shape, test_df.shape

((59476, 4), (14869, 4))

In [ ]:
train_df.to_csv("train.csv", columns=['token_description'], index=False, header=False)
test_df.to_csv("test.csv", columns=['token_description'], index=False, header=False)

In [ ]:
import fasttext

# Train the model
model = fasttext.train_supervised('train.csv')
result = model.test('test.csv')
result

(14795, 0.7839134842852314, 0.7839134842852314)

In [ ]:
# Predict labels for new data
text = 'stateof new jersey county camden subscribed sworn february pyeliza stasi affiant name weblarne ais lor maureen woollam notarys signatore sue notary public state new jersey commission expires maween woow notarys printed typed stamped name seal seal commission expires arr namsc docmagic page'
predicted_label = model.predict(text)
predicted_label

(('__label__21',), array([0.42527777]))

In [ ]:
df['token'].iloc[45]

93